In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark dynamics

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(6)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(np.diag((.5,.5)))

q = np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten()
v = np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten()
p = M.flatf(q,v.flatten())
print("q = ", q)
print("p = ", p)

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,qs) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))
M.plot()
M.plotx(qs)
plt.show()

# Langevin equations
see https://arxiv.org/abs/1605.09276

In [ ]:
from src.stochastics import Langevin
Langevin.initialize(M)

(ts,qps) = M.Langevinf(q,p,.15,.25,dWsf(M.dim.eval()))
M.plot()
M.plotx(qps[:,0,:])
plt.savefig('Langevin.pdf')
plt.show()

## Stochastic Development

In [ ]:
# development dynamics
from src.framebundle import FM
from src.stochastics import stochastic_development
FM.initialize(M)
stochastic_development.initialize(M)

n_steps.set_value(100)

In [ ]:
# landmarks
x = q
# frame
nu = np.linalg.cholesky(M.gsharpf(x))
# frame bundle element
u = np.concatenate((x,nu.flatten()))

In [ ]:
# stochastic development
w = dWsf(M.dim.eval()) # noise / anti-development
start = time.time()
(ts,us) = M.stochastic_developmentf(u,w)
print(time.time() - start)

# plot with frame
M.plot()
M.plotFMx(us,N_vec=5)
plt.show()

# plot only trajectory
plt.figure()
M.plot()
M.plotx(us[:,0:M.dim.eval()])
plt.show()

# plot noise / anti-development
plt.figure()
plt.plot(w[:,0],w[:,1])
plt.axis('equal')
plt.show()